# Download the buildings data

Question: How many buildings were mapped with RapID (Microsoft buildings) in selected region?

#### Import libs

In [1]:
import json
import os
import requests

### Get the data for every specified region

#### Definitions

In [2]:
# define the dictinaries

data_dic = {0: "blds_continent-states_18-23_ai.json",
            1: "blds_africa-states_18-23_ai.json",
            2: "blds_northamerica-states_18-23_ai.json",
            3: "blds_europe-states_18-23_ai.json",
            4: "blds_asia-states_18-23_ai.json", }

geojson_dic = {0: "geojson-continent-states",
                      1: "geojson-africa-states",
                      2: "geojson-northamerica-states",
                      3: "geojson-europe-states",
                      4: "geojson-asia-states",}

In [3]:
# define the user input

while True:
    try:
        user_d_spec = int(input("Which data do you want to download?\
                            Enter 0 for continents (2018–2023),\
                            1 for Africa states (2018–2023),\
                            2 for North-Central America states (2018–2023)\
                            3 for Europe states (2018–2023)\
                            4 for Asia states (2018–2023)"
                                ))
    except ValueError:
        print("Please enter a number.")
        continue
    if user_d_spec not in range(len(geojson_dic)):
        print("Please enter 0, 1, 2, 3 or 4.")
        continue
    break


In [11]:
# define the functions

def convert_geojson_structure(original_dict):
    new_dict = {}
    new_dict["type"] = "FeatureCollection"
    new_dict["features"] = []

    feature_dict = {}
    feature_dict["type"] = "Feature"
    feature_dict["geometry"] = {}
    feature_dict["geometry"]["type"] = "MultiPolygon"

    new_dict["features"].append(feature_dict)

    feature_dict["geometry"]["coordinates"] = original_dict["geometry"]["coordinates"]
    # feature_dict["geometry"]["type"] = "MultiPolygon"
    feature_dict["properties"] = original_dict["properties"]

    return new_dict

def data_gen(feature):

    desired_geojson_structure = convert_geojson_structure(feature)

    parameters = {
        # pass GeoJSON as string.
        "bpolys": json.dumps(desired_geojson_structure),
        "filter": "building=* and building!=no and geometry:polygon",
        "groupByKey": "source",
        "groupByValues": "microsoft/BuildingFootprints",
        "format": "json",
        "time": "2018-01-01/2023-01-01/P1Y",
        }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
        }

    for value in parameters.values():
        assert value != "", "Please provide values for the parameters"

    response = requests.post(url, data=parameters, headers=headers)
    response.raise_for_status()  # Raise an Exception if HTTP Status Code is not 200

    print("Response:")
    print(desired_geojson_structure["features"][0]["properties"]["NAME_EN"])
    print(json.dumps(response.json(), indent=4))  # Pretty print response

    result = response.json()["groupByResult"]

    for state in result:
        state["groupByObject"][0] = desired_geojson_structure["features"][0]["properties"]["NAME_EN"]

    return result

In [12]:
# define the URL

base_url = "https://api.ohsome.org/v1"
endpoint = "/elements/count/groupBy/boundary/groupBy/tag"
url = base_url + endpoint

#### Get the data

In [14]:
os.chdir('..')

with open(f"geojson-regions\\{geojson_dic[user_d_spec]}.geojson", "r") as file:
    bpolys = json.load(file)

data = []

for i, feature in enumerate(bpolys["features"][0:2]):
    data.append(data_gen(feature))


Response:
Ethiopia
{
    "attribution": {
        "url": "https://ohsome.org/copyrights",
        "text": "\u00a9 OpenStreetMap contributors"
    },
    "apiVersion": "1.9.0",
    "groupByResult": [
        {
            "groupByObject": [
                "feature1",
                "remainder"
            ],
            "result": [
                {
                    "timestamp": "2018-01-01T00:00:00Z",
                    "value": 284919.0
                },
                {
                    "timestamp": "2019-01-01T00:00:00Z",
                    "value": 350579.0
                },
                {
                    "timestamp": "2020-01-01T00:00:00Z",
                    "value": 395155.0
                },
                {
                    "timestamp": "2021-01-01T00:00:00Z",
                    "value": 470680.0
                },
                {
                    "timestamp": "2022-01-01T00:00:00Z",
                    "value": 629701.0
                },
     

In [19]:
# export the data

def export_data(name):
    with open(f"downloaded-data\\test_{name}", "w") as file:
        json.dump(data, file, indent=4)


export_data(data_dic[user_d_spec])
